In [1]:
import pandas as pd
import numpy as np
import requests
import json
import requests
import utils as ut

# Para traer la api key
import os
from dotenv import load_dotenv

In [2]:
# Importamos la api key
# Obtén la ruta absoluta del archivo .env
env_path = os.path.join(os.path.dirname('testing.ipynb'), '..', 'conf', '.env')

# Carga las variables de entorno desde el archivo .env
load_dotenv(env_path)

# Accede a la variable de entorno
key_admin = os.getenv('key_admin')

---

In [3]:
id = ut.datos_invocador(['GC Tincho'])['id']
id

Se finalizo el proceso de transformacion para GC Tincho.


0    y3gAnTO-EtujHRBDa8IOytuTLb1b_mq7f-MIqIarQEoBvQ
Name: id, dtype: object

In [4]:
def datos_invocador(nombres_invocador:list)->pd.DataFrame:
    
    # Definimos algunas variables
    key = key_admin
    region = 'LA2'
    
    # Creamos la variable para almacenar 
    df_personal_infomation = pd.DataFrame()
    
    # Creamos un bucle for para que funcionen la lista de nombres a emplear.
    for nombre_invocador in nombres_invocador:
        
        # Conectamos a la API
        url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{nombre_invocador}?api_key={key}' 
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error: no se pudo obtener la maestría de campeones del invocador {nombre_invocador}.")
            return None

        # Lo transformamos en un texto
        texto = response.text

        # Lo transformamos en un JSON y lo normalizamos en un pd.df
        json_info = json.loads(texto)
        personal_infomation = pd.json_normalize(json_info)

        # Almacenamos los datos
        df_personal_infomation = pd.concat([df_personal_infomation, personal_infomation], ignore_index=True)

        print(f'Se finalizo el proceso de transformacion para {nombre_invocador}.')
    
    return df_personal_infomation

In [5]:
# Creamos la funcion para que nos de los datos de la partida en viv
    
def live_match(nombre_jugador:str)->pd.DataFrame:
    
    #definimos algunas variables
    key = key_admin
    region = 'la2'
        
    # Creamos el df para almacenar los datos
    df_match = pd.DataFrame()
    
    # Obtenemos el id del jugador
    id = datos_invocador([nombre_jugador])['id'][0]
    print(f'El id del jugador es: {id}')
    
    # Conexion a la API
    url = f'https://{region}.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{id}?api_key={key_admin}'
    response = requests.get(url)
    if response.status_code != 200:
        print(f'Error al obtener datos de: {nombre_jugador}')
        return None
    
    # Lo transformamos en texto 
    texto = response.text
    
    # Lo transformamos en JSON y normalziamos con pd
    json_info = json.loads(texto)
    df_match = pd.json_normalize(json_info)
    
    print(f'Se finalizo el proceso para la partida de {nombre_jugador}')
    
        # Creamos el df vacio
    detalle_partida = pd.DataFrame()

    # Creamos el bucle para normalizar cada JSON    
    for i in range(0,10,1):
        
        # Normalizamos cada jugador
        player_normalize = pd.json_normalize(pd.json_normalize(df_match['participants'])[i])
        
        # Guardamos los datos
        detalle_partida = pd.concat([detalle_partida, player_normalize])

    return detalle_partida

In [20]:
# Creamos el df vacio
detalle_partida = pd.DataFrame()

# Creamos el bucle para normalizar cada JSON    
for i in range(0,10,1):
    
    # Normalizamos cada jugador
    player_normalize = pd.json_normalize(pd.json_normalize(partida['participants'])[i])
    
    # Guardamos los datos
    detalle_partida = pd.concat([detalle_partida, player_normalize])

    
detalle_partida

,teamId,spell1Id,spell2Id,championId,profileIconId,summonerName,bot,summonerId,gameCustomizationObjects,perks.perkIds,perks.perkStyle,perks.perkSubStyle
0,100,4,14,99,5736,Foortune,False,55MEpu53V7U8jG3j_umw8AuBPQAGpaj7EA7D75inMfyGxqM,[],"[8229, 8226, 8210, 8237, 8126, 8106, 5008, 500...",8200,8100
0,100,4,12,150,4864,M1KE TYS0N,False,FP95A4NcwZsbHYKATDFJpAMv8wzOpyxHxnfkadid94XNeCQ,[],"[8010, 9111, 9104, 8299, 8473, 8451, 5005, 500...",8000,8400
0,100,4,14,518,5434,AntonyKiller77,False,VxqE0JuZi21SybY9oHea7IRvliueUMhz4sKn5cTZeCDDS2...,[],"[8112, 8139, 8138, 8135, 8226, 8237, 5005, 500...",8100,8200
0,100,4,11,121,5755,PermAcànc3r,False,nU5_o77OIGTyHeMPNLC2mQ1L3khgtloykhzXa4HAuGHwuf...,[],"[8369, 8304, 8321, 8347, 8143, 8135, 5008, 500...",8300,8100
0,100,7,4,81,11,peneriko123,False,8s9RcWQHEFrV4BjRNloqMVa8I5LCSGknzuzTuCM_cp64YS...,[],"[8005, 8009, 9103, 8014, 8345, 8304, 5005, 500...",8000,8300
0,200,4,3,143,5840,Ptrick,False,Jzh7TLAzGb2dg5uYZdqUpvUHqTx114WHRb4M19s01oLWGb...,[],"[8229, 8226, 8210, 8237, 8345, 8347, 5008, 500...",8200,8300
0,200,4,11,111,986,GC Crotolamo,False,500-z1qVQ8GwiEHubwYokUDR5dhy5OSePjRTNBdnm3OK6w,[],"[8439, 8401, 8473, 8242, 8345, 8347, 5007, 500...",8400,8300
0,200,12,4,126,3501,GC Ca Cdos 601,False,0nPz7ri619KkdYlJiPSnsVS35NgxoQtJoWH_2wcjYRSLRA,[],"[8230, 8226, 8210, 8236, 8009, 8014, 5005, 500...",8200,8000
0,200,7,4,22,5031,Magnuskid,False,8HoShGdX3uGMwj5mMr2uLDcnd7etdDMK4o4UX6Ut0Keu,[],"[8008, 9111, 9104, 8014, 8126, 8106, 5005, 500...",8000,8100
0,200,4,14,17,5840,Tiburoncionuhaha,False,CtbCzfJHIBPDKOOpTSEbREyrISbmP8F_dXVKVtOxwtHv,[],"[8437, 8446, 8473, 8451, 8126, 8106, 5005, 500...",8400,8100


In [9]:
partida02 = live_match('Sir Vladimir')

partida02

Se finalizo el proceso de transformacion para Sir Vladimir.
El id del jugador es: rP8a5W6eehGqLQsu2TxwtYLViJYbW9Fs8rL16eXovQrzng
Se finalizo el proceso para la partida de Sir Vladimir


,teamId,spell1Id,spell2Id,championId,profileIconId,summonerName,bot,summonerId,gameCustomizationObjects,perks.perkIds,perks.perkStyle,perks.perkSubStyle
0,100,4,14,86,1130,Sir Vladimir,False,rP8a5W6eehGqLQsu2TxwtYLViJYbW9Fs8rL16eXovQrzng,[],"[8010, 9111, 9105, 8299, 8446, 8429, 5008, 500...",8000,8400
0,100,1,4,119,28,nyhzz0k4p,False,ZciDWHMMmWNlssrGlyE2WsKDlaS4OzWuRhngOUKJF_Qu7Q,[],"[8008, 9111, 9103, 8299, 8138, 8135, 5005, 500...",8000,8100
0,100,12,4,517,5637,Consumo Gusto,False,9yfntU8YHnOJF41Zp89ulKrB268LJVJZgwIGJRfi-jH_eN...,[],"[8010, 8009, 9105, 8299, 8444, 8242, 5008, 500...",8000,8400
0,100,11,4,121,3890,FeVsm,False,kW7hP5-7D6gKZRX6V6CR9aQ56jcLZWg80dpIiUuDlZwRNA,[],"[8369, 8304, 8321, 8347, 8143, 8135, 5008, 500...",8300,8100
0,100,14,4,25,5765,ElMataSuegras,False,-DRkS-LVXdNtAAGSo4cW1UvoKDdW6gHXUqYrzuUo3kC82g,[],"[8229, 8226, 8210, 8237, 8345, 8313, 5008, 500...",8200,8300
0,200,4,14,201,5692,D4Ss,False,VEf_yJg3_meetiOjdgOBEEbtWlWCu7v4jqch6vO0KTgzsA,[],"[8465, 8463, 8473, 8242, 8345, 8347, 5005, 500...",8400,8300
0,200,4,14,99,9,juankalfnm,False,USDDyLfAEiJvsOv2Kyx1Fo3LQHfhh4LY6iobGcQEgUsg8q...,[],"[8229, 8275, 8210, 8236, 8139, 8138, 5007, 500...",8200,8100
0,200,4,3,157,3859,GSS,False,nAlHAGXKq1q6gwNuf_E1W4wzDfMB1-aEmH2UwIoPxIQyXw,[],"[8008, 9111, 9104, 8014, 8139, 8138, 5005, 500...",8000,8100
0,200,11,4,203,4661,Warning Peluca,False,OwFqL4r-4sQ4FJUSpZzpmwVzSbCQwAhh4CBWuEIUvGVCdA,[],"[8005, 9111, 9104, 8014, 8143, 8135, 5005, 500...",8000,8100
0,200,4,12,31,4068,ZepeUy,False,Ax97g9AkAbLdooAZNKbMi5XkOBx3BiQyatun2CpcwUY,[],"[8437, 8446, 8429, 8451, 9105, 9111, 5005, 500...",8400,8000


In [7]:
partida01 = pd.read_csv('live_match.csv', index_col=0)
partida01

,teamId,spell1Id,spell2Id,championId,profileIconId,summonerName,bot,summonerId,gameCustomizationObjects,perks.perkIds,perks.perkStyle,perks.perkSubStyle
0,100,4,14,99,5736,Foortune,False,55MEpu53V7U8jG3j_umw8AuBPQAGpaj7EA7D75inMfyGxqM,[],"[8229, 8226, 8210, 8237, 8126, 8106, 5008, 500...",8200,8100
0,100,4,12,150,4864,M1KE TYS0N,False,FP95A4NcwZsbHYKATDFJpAMv8wzOpyxHxnfkadid94XNeCQ,[],"[8010, 9111, 9104, 8299, 8473, 8451, 5005, 500...",8000,8400
0,100,4,14,518,5434,AntonyKiller77,False,VxqE0JuZi21SybY9oHea7IRvliueUMhz4sKn5cTZeCDDS2...,[],"[8112, 8139, 8138, 8135, 8226, 8237, 5005, 500...",8100,8200
0,100,4,11,121,5755,PermAcànc3r,False,nU5_o77OIGTyHeMPNLC2mQ1L3khgtloykhzXa4HAuGHwuf...,[],"[8369, 8304, 8321, 8347, 8143, 8135, 5008, 500...",8300,8100
0,100,7,4,81,11,peneriko123,False,8s9RcWQHEFrV4BjRNloqMVa8I5LCSGknzuzTuCM_cp64YS...,[],"[8005, 8009, 9103, 8014, 8345, 8304, 5005, 500...",8000,8300
0,200,4,3,143,5840,Ptrick,False,Jzh7TLAzGb2dg5uYZdqUpvUHqTx114WHRb4M19s01oLWGb...,[],"[8229, 8226, 8210, 8237, 8345, 8347, 5008, 500...",8200,8300
0,200,4,11,111,986,GC Crotolamo,False,500-z1qVQ8GwiEHubwYokUDR5dhy5OSePjRTNBdnm3OK6w,[],"[8439, 8401, 8473, 8242, 8345, 8347, 5007, 500...",8400,8300
0,200,12,4,126,3501,GC Ca Cdos 601,False,0nPz7ri619KkdYlJiPSnsVS35NgxoQtJoWH_2wcjYRSLRA,[],"[8230, 8226, 8210, 8236, 8009, 8014, 5005, 500...",8200,8000
0,200,7,4,22,5031,Magnuskid,False,8HoShGdX3uGMwj5mMr2uLDcnd7etdDMK4o4UX6Ut0Keu,[],"[8008, 9111, 9104, 8014, 8126, 8106, 5005, 500...",8000,8100
0,200,4,14,17,5840,Tiburoncionuhaha,False,CtbCzfJHIBPDKOOpTSEbREyrISbmP8F_dXVKVtOxwtHv,[],"[8437, 8446, 8473, 8451, 8126, 8106, 5005, 500...",8400,8100


In [10]:
partida03 = live_match('Angeletto')
partida03

Se finalizo el proceso de transformacion para Angeletto.
El id del jugador es: Lhdml6StgmOzBAge_T7gkMJsaHr9SpUXPtClL2YeU4475jI
Se finalizo el proceso para la partida de Angeletto


,teamId,spell1Id,spell2Id,championId,profileIconId,summonerName,bot,summonerId,gameCustomizationObjects,perks.perkIds,perks.perkStyle,perks.perkSubStyle
0,100,4,12,31,4030,ElPezRadamantis,False,XbgKbY-xxHMqOAp13calFVowhnbt96y_H9AfusxGU1jHQXo,[],"[8229, 8226, 8210, 8237, 8126, 8106, 5008, 500...",8200,8100
0,100,4,3,67,539,XxSk3ithxX,False,EOwd9qJnIDUOl05bAdozPlffJ-0PUyk79Z9apoUjjQg,[],"[8005, 9111, 9104, 8017, 8135, 8143, 5005, 500...",8000,8100
0,100,3,4,16,839,La Manuela,False,YTn-bfgpKzx3fKiP7L2K8wN6xuIlruWxDnqmqbUJ52Xszg,[],"[8214, 8226, 8210, 8237, 8473, 8453, 5008, 500...",8200,8400
0,100,4,12,150,3583,Angeletto,False,Lhdml6StgmOzBAge_T7gkMJsaHr9SpUXPtClL2YeU4475jI,[],"[8021, 9111, 9104, 8299, 8473, 8451, 5005, 500...",8000,8400
0,100,11,4,245,5414,Vitamina B,False,cSxv5knurIMAHglXz_FcR5v_ls9t_ilesOwncAvFzD48Ew,[],"[8128, 8143, 8138, 8135, 9105, 9111, 5005, 500...",8100,8000
0,200,4,7,81,5493,Tomy Shelby,False,_zEOA2TEygfmhlhwRIuNpgm0llU6_6kpGGXTB_G7fKYfHg,[],"[8369, 8304, 8345, 8347, 8226, 8210, 5005, 500...",8300,8200
0,200,11,4,517,29,ChiboloM3epo,False,cMpqHAkNGy_v_6C4pOT1e14AXXlrAhoIkfvzdJ-t0Ls1jH...,[],"[8128, 8143, 8138, 8135, 8347, 8304, 5008, 500...",8100,8300
0,200,4,14,84,3587,1412Kudo,False,vEjsvoIzRQqIaAAF5VuseWhBwrZ1WgCWbF7OQ3Eqex9UMw,[],"[8112, 8143, 8138, 8105, 8242, 8473, 5008, 500...",8100,8400
0,200,12,6,75,4762,26 DE JUNIO 2011,False,07BDftjlHf8g4lvIhn2xNl-736cdxX_OXMuG3D-pyiPQkVM,[],"[8021, 9111, 9105, 8299, 8444, 8242, 5007, 500...",8000,8400
0,200,14,4,235,5021,SkrfaceTM,False,e-30knHIWIzBv9Z-tDMoJq3T61px1wcHVBmScCqXwgPraEg,[],"[8021, 8009, 9104, 8017, 8473, 8453, 5005, 500...",8000,8400


In [8]:
partida04 = live_match('Klenra 742')
partida04

Se finalizo el proceso de transformacion para Klenra 742.
El id del jugador es: 8Ad_jK7v9dpCsIO1ncpqsJ0FLxphZY1AH96Eb9GWSOztzQ
Se finalizo el proceso para la partida de Klenra 742


,teamId,spell1Id,spell2Id,championId,profileIconId,summonerName,bot,summonerId,gameCustomizationObjects,perks.perkIds,perks.perkStyle,perks.perkSubStyle
0,100,11,4,78,1232,Klenra 742,False,8Ad_jK7v9dpCsIO1ncpqsJ0FLxphZY1AH96Eb9GWSOztzQ,[],"[8128, 8126, 8138, 8105, 8234, 8232, 5008, 500...",8100,8200
0,100,3,4,201,5426,Gabox 360,False,RMzx7O0j8mAwb-7OeOXIuJ4-hDpWYPFFw6e9ZMpI2k7RsQ,[],"[8351, 8304, 8321, 8410, 8429, 8463, 5007, 500...",8300,8400
0,100,4,7,22,4555,dangerfury,False,o2vdNUl2W-r_ZAN7hYqizaXLxlZYN-wt3snXar5VHCJkAw,[],"[8008, 9111, 9103, 8014, 8126, 8106, 5005, 500...",8000,8100
0,100,4,12,54,5814,Pocoio,False,6vlwofsXzClFou-OqE6VNTSDvCjFd3lrMgi7fEs_QfYOJQ,[],"[8437, 8401, 8429, 8451, 8236, 8226, 5005, 500...",8400,8200
0,100,14,4,1,654,LaosHi,False,5gsGGDJvCWBff1MWP_EUyHeTU1x88pDPEdZ85Senp0fG,[],"[8112, 8126, 8138, 8106, 8233, 8226, 5008, 500...",8100,8200
0,200,4,11,79,3791,Yoel Tulas,False,ZNTTe7uyIfPXzCDrU5jIq2RUjqHHdGIrnWxyr9MueHy_lw,[],"[8128, 8143, 8138, 8135, 8210, 8232, 5008, 500...",8100,8200
0,200,4,12,5,3013,Yoel Rabos,False,k3S7bqSiDIDrWQqc3z5pezewyJ4zd72yYGrNLcMRqlbEeA,[],"[8008, 9111, 9105, 8299, 8242, 8444, 5005, 500...",8000,8400
0,200,4,3,99,1666,Yoel Biza,False,m0v0tgtQQ8mQ2xC9CcO12lndKeHuiCEav3vIrshvODiZXw,[],"[8128, 8126, 8138, 8106, 8226, 8210, 5008, 500...",8100,8200
0,200,4,3,103,3789,Yoel Gucci,False,1GyuujaGEQDLjvOc7E5GzrGlAvdEW5dQJEZd8e-PEbIZuA,[],"[8214, 8226, 8210, 8237, 8345, 8347, 5005, 500...",8200,8300
0,200,6,4,51,4550,ł C A R D E ł,False,rIk05pc3j3RCH0Sxta-RkHsDNf5jxJmWX3By01LPtlTx,[],"[8008, 8009, 9104, 8014, 8139, 8105, 5005, 500...",8000,8100


In [11]:
# Guardamos todas las partidas

partida01.to_csv(path_or_buf='live_matchs/partida01.csv', sep=';', encoding='UTF-8')
partida02.to_csv(path_or_buf='live_matchs/partida02.csv', sep=';', encoding='UTF-8')
partida03.to_csv(path_or_buf='live_matchs/partida03.csv', sep=';', encoding='UTF-8')
partida04.to_csv(path_or_buf='live_matchs/partida04.csv', sep=';', encoding='UTF-8')